# K-En Yakın Komşu (KNN) 

Tahminler gözlem benzerliğine göre yapılır.

### KNN Basamakları
* Komşu sayısını belirle (K)
* Bilinmeyen nokta ile diğer tüm noktalar ile arasındaki uzaklıkları hesapla
* Uzaklıkları sırala ve belirlenen k sayısına göre en yakın olan k gözlemi seç
* Sınıflandırma ise en sık sınıfı, regresyon ise ortalama değeri tahmin değeri olarak ver.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt 
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor

import warnings
warnings.filterwarnings('ignore')

## Veri Önişleme

In [2]:
hitters = pd.read_csv('Hitters.csv')
df = hitters.copy()
df = df.dropna()
dummies = pd.get_dummies(df.select_dtypes(["object"])) 
y = df['Salary']
X_ = df.drop(['Salary', 'League', 'NewLeague', 'Division'], axis=1).astype("float64")
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Model

In [3]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [4]:
knn_model.n_neighbors

5

## Tahmin

In [5]:
y_pred = knn_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

426.6570764525201

In [6]:
RMSE = []

for k in range(10):
    k += 1
    knn_model = KNeighborsRegressor(n_neighbors=k).fit(X_train,  y_train)
    y_pred = knn_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_pred, y_test))
    RMSE.append(rmse)
    print("k =", k, "için rmse değeri: ", rmse)

k = 1 için rmse değeri:  455.03925390751965
k = 2 için rmse değeri:  415.99629571490965
k = 3 için rmse değeri:  420.6765370082348
k = 4 için rmse değeri:  428.8564674588792
k = 5 için rmse değeri:  426.6570764525201
k = 6 için rmse değeri:  423.5071669008732
k = 7 için rmse değeri:  414.9361222421057
k = 8 için rmse değeri:  413.7094731463598
k = 9 için rmse değeri:  417.84419990871265
k = 10 için rmse değeri:  421.6252180741266


## Tuning

In [7]:
knn_params = {
    'n_neighbors': np.arange(1,30,1)
}

knn = KNeighborsRegressor()

knn_cv = GridSearchCV(knn, knn_params, cv=10)

In [8]:
knn_cv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [9]:
knn_cv.best_params_

{'n_neighbors': 8}

In [10]:
RMSE = []
RMSE_CV = []

for k in range(10):
    k += 1
    knn_model = KNeighborsRegressor(n_neighbors=k).fit(X_train,  y_train)
    y_pred = knn_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_pred, y_test))
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model, X_train, y_train, cv=10, scoring = "neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    print("k =", k, "\t için rmse değeri: ", rmse, "\t rms_cv değeri: ", rmse_cv)

k = 1 	 için rmse değeri:  455.03925390751965 	 rms_cv değeri:  325.39475147063825
k = 2 	 için rmse değeri:  415.99629571490965 	 rms_cv değeri:  293.24000183333817
k = 3 	 için rmse değeri:  420.6765370082348 	 rms_cv değeri:  283.7486667487823
k = 4 	 için rmse değeri:  428.8564674588792 	 rms_cv değeri:  286.3240222024089
k = 5 	 için rmse değeri:  426.6570764525201 	 rms_cv değeri:  290.0705466132226
k = 6 	 için rmse değeri:  423.5071669008732 	 rms_cv değeri:  298.1263115575851
k = 7 	 için rmse değeri:  414.9361222421057 	 rms_cv değeri:  294.77070479194987
k = 8 	 için rmse değeri:  413.7094731463598 	 rms_cv değeri:  291.98672028891235
k = 9 	 için rmse değeri:  417.84419990871265 	 rms_cv değeri:  295.7162739573105
k = 10 	 için rmse değeri:  421.6252180741266 	 rms_cv değeri:  301.31047022701154


In [11]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv.best_params_['n_neighbors'])

In [12]:
knn_tuned.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=8)

In [13]:
y_pred = knn_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

413.7094731463598

## Kaynaklar

### Kurslar
* https://www.udemy.com/course/python-egitimi/